In [2]:
import numpy as np

In [3]:
y = [np.array([2603.77028549, 2665.292078  , 2251.261876  , 2510.15775133]),
 np.array([1558.72110143, 1728.68097843, 1865.72775535, 1413.52330867,
        1158.81381311, 1691.86795088]),
 np.array([1527.73185301, 1443.51857169, 1041.14137785])]

In [6]:
for g in range(3):
    y[g] = np.round(y[g]).astype(int)

In [7]:
y

[array([2604, 2665, 2251, 2510]),
 array([1559, 1729, 1866, 1414, 1159, 1692]),
 array([1528, 1444, 1041])]

In [10]:
N = sum([len(yg) for yg in y])

In [11]:
N

13

In [13]:
G=3

In [25]:
n = [len(yg) for yg in y]

In [26]:
n

[4, 6, 3]

In [35]:
mubar = [np.mean(yg) for yg in y]
print(mubar)

[2507.5, 1569.8333333333333, 1337.6666666666667]


In [33]:
vbar = np.sum([np.sum((y[g] - mubar[g])**2) for g in range(G)]) / N
print(vbar)

42816.96153846154


### MAP+Laplace

In [31]:
sigma2_MAP = N * vbar / (N+2)
print(sigma2_MAP)

37108.03333333333


In [29]:
stdL_mu = [np.sqrt(vbar / (N+2) * N / ng) for ng in n]
print(stdL_mu)

[96.31722760406538, 78.64268702315698, 111.2175545696112]


In [34]:
stdL_sigma2 = np.sqrt(2/(N+2)) * N * vbar / (N+2)
print(stdL_sigma2)

13549.93794754686


### MCMC

In [36]:
import pystan

In [38]:
model_code = """
data {
    int<lower=1> G;
    int<lower=1> N;
    int<lower=1, upper=G> label[N];  // group label
    real y[N];
}
parameters {
    real mu[G];
    real<lower=0> sigma2;
}
model {
    target += -log(sigma2);
    for (j in 1:N){
        target += normal_lpdf(y[j] | mu[label[j]], sqrt(sigma2));
    }
}
"""
model = pystan.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_da50ab0c023b335e73b0fc17ba905872 NOW.


In [54]:
data = {
    'G': G,
    'N': N,
    'y': np.concatenate(y) ,
    'label': np.concatenate([[g+1] * n[g] for g in range(G)])
}

In [55]:
data

{'G': 3,
 'N': 13,
 'y': array([2604, 2665, 2251, 2510, 1559, 1729, 1866, 1414, 1159, 1692, 1528,
        1444, 1041]),
 'label': array([1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3])}

In [56]:
fit = model.sampling(data, seed=123456)

In [57]:
samples = fit.extract()

In [63]:
np.mean(samples['mu'], axis=0)

array([2507.27700403, 1570.27989673, 1340.45050983])

In [64]:
np.median(samples['mu'], axis=0)

array([2510.18135758, 1570.86572614, 1339.41578624])

In [65]:
np.std(samples['mu'], axis=0)

array([131.42099704, 107.04966321, 148.93665698])

In [66]:
np.mean(samples['sigma2'])

69487.9137985119

In [68]:
np.median(samples['sigma2'])

59354.86193979092

In [67]:
np.std(samples['sigma2'])

40098.543439842506

### Exact

In [71]:
std_mu = np.sqrt(vbar / (N-5) * N/np.array(n))
print(std_mu)

[131.88779559 107.68593416 152.29090857]


In [72]:
mubar

[2507.5, 1569.8333333333333, 1337.6666666666667]

In [74]:
std_sigma2 = np.sqrt(2 / (N-7)) * N * vbar / (N-5)
print(std_sigma2)

40170.62443893301


In [77]:
mean_sigma2 = N*vbar / (N-5)
print(mean_sigma2)

69577.5625


In [78]:
mode_sigma2 = N*vbar / (N)
print(mode_sigma2)

42816.96153846154
